In [17]:
import spacy
import requests
import os
from google.colab import userdata
from huggingface_hub import InferenceClient
#from dotenv import load_dotenv
# Load environment variables from .env file
#load_dotenv()
#api_key = os.getenv("HUGGINGFACE_API_KEY")
api_key = userdata.get("HUGGINGFACE_API_KEY")
nlp = spacy.load("en_core_web_sm")
api_url = "https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"





In [18]:
def pre(text):
    doc = nlp(text)
    return "".join(token.text+" " for token in doc )


print(pre("I am a student."))

I am a student . 


In [19]:
def payload(doc):
    headers = {
    "Authorization":f"Bearer {api_key}",
    }
    payload = {
    "messages": [

        {
            "role": "user",
            "content": doc
        }
    ],
    "model": "mistralai/Mistral-7B-Instruct-v0.1",
    }

    response = requests.post(api_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]

    else:
        print(f"Error: {response.status_code}")
        print(f"Error: {response.text}")
        return None


In [20]:
def main():

    i = 0
    while i != 1:
        text = input("Your turn:")
        if text == "exit":
          i=1
          break
        else:
            ll = pre(text)
            print(payload(ll))



Recording audio from browser and uploading it


In [25]:
from IPython.display import display, HTML

display(HTML('''
<h3>🎤 Record your voice</h3>
<button onclick="startRecording()">Start Recording</button>
<button onclick="stopRecording()">Stop & Upload</button>
<p id="status"></p>
<script>
let mediaRecorder;
let audioChunks;

async function startRecording() {
    document.getElementById("status").innerText = "Recording...";
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    mediaRecorder = new MediaRecorder(stream);
    audioChunks = [];

    mediaRecorder.ondataavailable = event => {
        audioChunks.push(event.data);
    };

    mediaRecorder.onstop = async () => {
        const blob = new Blob(audioChunks, { type: 'audio/wav' });
        const file = new File([blob], "recorded.wav", { type: "audio/wav" });

        const dataTransfer = new DataTransfer();
        dataTransfer.items.add(file);

        const input = document.createElement('input');
        input.type = 'file';
        input.files = dataTransfer.files;
        input.style.display = 'none';
        document.body.appendChild(input);
        input.click();
    };

    mediaRecorder.start();
}

function stopRecording() {
    mediaRecorder.stop();
    document.getElementById("status").innerText = "Recording stopped. Upload initiated.";
}
</script>
'''))


In [26]:
from google.colab import files
uploaded = files.upload()


In [21]:
client = InferenceClient(
    provider="fal-ai",
    api_key= api_key,
)

output = client.automatic_speech_recognition("sample1.flac", model="openai/whisper-large-v3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'sample1.flac'